# 1. Load the libraries

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import cv2
import glob
import numpy as np
import pandas as pd  
from PIL import Image
from skimage import measure
from keras import backend as K
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.patches as patches
import tensorflow as tf
from skimage.measure import label, regionprops, regionprops_table
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Flatten, Dense, GlobalAveragePooling2D, MaxPooling2D, AveragePooling2D
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.efficientnet import preprocess_input


2024-11-25 21:11:11.402387: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-25 21:11:12.411186: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# 2. EfficientNetB0

## 2.1 Read images and compute 99% quantile

In [2]:
#define the file paths for the images and corresponding masks
image_path = '/4tbint/Corrected Merged FOVs/Set5/20200408_N7863__2020-04-08T10_57_43-Measurement 2'
mask_path = '/4tbint/Cellpose Masks/Set5/20200408_N7863__2020-04-08T10_57_43-Measurement 2'

#use the glob library to generate lists of image and mask filenames
images =  sorted([os.path.basename(x) for x in glob.glob(image_path + '/*.tiff')])
cellpose_mask = sorted([os.path.basename(x) for x in glob.glob(mask_path + '/*.tiff')])

ch1 = [] #fibroblast
ch2 = [] #cancer
ch4 = [] #dapi

for i in range(0, 1152, 3):
    
    CH1_path = os.path.join(image_path, images[i])
    CH2_path = os.path.join(image_path, images[i+2]) #for MHB ch3 is cancer 
    CH4_path = os.path.join(image_path, images[i+1]) #for MHB ch2 is DAPI

    #print(CH1_path)
    #print(CH2_path)
    #print(CH4_path)

    #read images and masks using the Image library, and converts them to numpy arrays.
    ch1_img = Image.open(CH1_path)
    ch2_img = Image.open(CH2_path)
    ch4_img = Image.open(CH4_path)

    ch1.append(np.array(ch1_img))
    ch2.append(np.array(ch2_img))
    ch4.append(np.array(ch4_img))

ch1_max = np.max(ch1)
ch2_max = np.max(ch2)
ch4_max = np.max(ch4)
print("Maximum intensity for channel 1:", ch1_max)
print("Maximum intensity for channel 2:", ch2_max)
print("Maximum intensity for channel 4:", ch4_max)

ch1_q099 =  np.quantile(ch1, 0.99)
ch2_q099 =  np.quantile(ch2, 0.99)
ch4_q099 =  np.quantile(ch4, 0.99)
print("Quantile_099 channel 1:", ch1_q099)
print("Quantile_099 channel 2:", ch2_q099)
print("Quantile_099 channel 4:", ch4_q099)

del ch1, ch2, ch4

Maximum intensity for channel 1: 65289
Maximum intensity for channel 2: 65535
Maximum intensity for channel 4: 64458
Quantile_099 channel 1: 1959.0
Quantile_099 channel 2: 2514.0
Quantile_099 channel 4: 5185.0


## 2.2 Load the model and Read images as tensors and extract features

In [ ]:
import tensorflow as tf

# Enable memory growth for GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("Logical devices:", tf.config.list_logical_devices())
        
with tf.device('/CPU:0'):
    # Load without compiling
    model = load_model('EfficientNetB0_trained_binary_MHB.h5',custom_objects={
        'weighted_custom_loss': lambda y_true, y_pred: weighted_custom_loss(y_true, y_pred, class_weights_list)},
    compile=False)  # Avoid attempting to compile with the custom loss
    layer = model.get_layer(name="top_conv").output
    output = GlobalAveragePooling2D()(layer)
    feature_extraction_model = Model(inputs=model.inputs, outputs=output)
    print("Model loaded successfully.")

# Use MirroredStrategy to automatically distribute across GPUs 0 and 1
strategy = tf.distribute.MirroredStrategy()
print(f"Number of devices: {strategy.num_replicas_in_sync}")

# All the code inside this scope will be distributed across both GPUs
with strategy.scope():

    @tf.function
    # Example of preprocessing and extracting features
    def preprocess_image(image):
        image = preprocess_input(image)  # Apply EfficientNet-specific preprocessing
        return image
    @tf.function
    def extract_features(image):
        preprocessed_image = preprocess_image(image)
        features = feature_extraction_model(preprocessed_image)
        return features

    #define the file paths for the images and corresponding masks
    image_path = '/4tbint/Corrected Merged FOVs/Set5/20200408_N7863__2020-04-08T10_57_43-Measurement 2'
    mask_path = '/4tbint/Cellpose Masks/Set5/20200408_N7863__2020-04-08T10_57_43-Measurement 2'

    #use the glob library to generate lists of image and mask filenames
    images =  sorted([os.path.basename(x) for x in glob.glob(image_path + '/*.tiff')])
    cellpose_mask = sorted([os.path.basename(x) for x in glob.glob(mask_path + '/*.tiff')])

    # "mean_feature_list" and "j" variables are initialized to empty list and 0, respectively to store the features for each image and to iterate over the list of masks.
    mean_fetaure_list =[]
    image_list = []
    j=0

    #loop with step size of 3 to iterate over a range of indices, pulling the filenames for three channels of each image.
    for i in range(0, 1152, 3):

        CH1_path = os.path.join(image_path, images[i]) #fibroblast
        CH2_path = os.path.join(image_path, images[i+2]) #cancer
        CH4_path = os.path.join(image_path, images[i+1]) #dapi
        
        #read images and masks using the Image library, and converts them to numpy arrays.
        ch1_img = Image.open(CH1_path)
        ch2_img = Image.open(CH2_path)
        ch4_img = Image.open(CH4_path)

        image_array_ch1 = np.array(ch1_img)
        image_array_ch2 = np.array(ch2_img)
        image_array_ch4 = np.array(ch4_img)

        #use cellpose mask to extract individual cell regions,to generate 50x50 pixel images.
        cellpose_path = os.path.join(mask_path, cellpose_mask[j])
        cellpose_img = Image.open(cellpose_path)
        masks_ch2 = np.array(cellpose_img)
        labels_ch2 = label(masks_ch2)
        props = regionprops(labels_ch2)

        j=j+1
        single_cell_bbox = []

        for cell_label, target_region in enumerate(props, start=1):
            centroid_row, centroid_col = target_region.centroid

            half_size = 45
            min_row = int(max(centroid_row - half_size, 0))
            max_row = int(min(centroid_row + half_size, image_array_ch2.shape[0]))
            min_col = int(max(centroid_col - half_size, 0))
            max_col = int(min(centroid_col + half_size, image_array_ch2.shape[1]))

            if (min_row > 0 and max_row < image_array_ch2.shape[0] and 
            min_col > 0 and max_col < image_array_ch2.shape[1]):

                target_cell_mask = labels_ch2 == cell_label

                cell_area_original = np.where(target_cell_mask[min_row:max_row, min_col:max_col], image_array_ch2[min_row:max_row, min_col:max_col], 0)

                empty_array_ch2 = np.zeros((90, 90))
                paste_row_original = (90 - cell_area_original.shape[0]) // 2
                paste_col_original = (90 - cell_area_original.shape[1]) // 2

                empty_array_ch2[paste_row_original:paste_row_original+cell_area_original.shape[0], paste_col_original:paste_col_original+cell_area_original.shape[1]] = cell_area_original

                empty_array_cellpose = np.zeros((90, 90))
                cell_area_cellpose = target_cell_mask[min_row:max_row, min_col:max_col]

                paste_row_cellpose = (90 - cell_area_cellpose.shape[0]) // 2
                paste_col_cellpose = (90 - cell_area_cellpose.shape[1]) // 2

                empty_array_cellpose[paste_row_cellpose:paste_row_cellpose+cell_area_cellpose.shape[0], paste_col_cellpose:paste_col_cellpose+cell_area_cellpose.shape[1]] = cell_area_cellpose

                cell_area_ch1 = np.where(target_cell_mask[min_row:max_row, min_col:max_col], image_array_ch1[min_row:max_row, min_col:max_col], 0)
                cell_area_ch4 = np.where(target_cell_mask[min_row:max_row, min_col:max_col], image_array_ch4[min_row:max_row, min_col:max_col], 0)

                empty_array_ch1 = np.zeros((90, 90))
                paste_row_ch1 = (90 - cell_area_ch1.shape[0]) // 2
                paste_col_ch1 = (90 - cell_area_ch1.shape[1]) // 2

                empty_array_ch4 = np.zeros((90, 90))
                paste_row_ch4 = (90 - cell_area_ch4.shape[0]) // 2
                paste_col_ch4 = (90 - cell_area_ch4.shape[1]) // 2

                empty_array_ch1[paste_row_ch1:paste_row_ch1+cell_area_ch1.shape[0], paste_col_ch1:paste_col_ch1+cell_area_ch1.shape[1]] = cell_area_ch1
                empty_array_ch4[paste_row_ch4:paste_row_ch4+cell_area_ch4.shape[0], paste_col_ch4:paste_col_ch4+cell_area_ch4.shape[1]] = cell_area_ch4

                empty_array_ch1 = (empty_array_ch1.astype('float32')/ch1_q099)*255
                empty_array_ch2 = (empty_array_ch2.astype('float32')/ch2_q099)*255
                empty_array_ch4 = (empty_array_ch4.astype('float32')/ch4_q099)*255

                singlecell = np.stack((empty_array_ch2, empty_array_ch1, empty_array_ch4), axis=-1)
                reshaped_singlecell = singlecell.reshape((1,90,90,3))
                single_cell_bbox.append(reshaped_singlecell)
                #print(np.max(empty_array_ch1))

        single_cells = np.array(single_cell_bbox)

        if len(single_cells) != 0:

            print(images[i+2])

            #convert single cell objects into a tensor using tf.data.Dataset.from_tensor_slices.
            single_cell_tensor = tf.data.Dataset.from_tensor_slices(single_cells)

            #A map operation is used to apply a feature extraction function, extract_features, to each cell image 
            #using a pre-trained model. This creates a feature_dataset of extracted features.
            feature_dataset = single_cell_tensor.map(extract_features)

            #features are summed across all cells, divided by the number of cells, and added to the mean_feature_list.
            sum_feature_dataset = tf.zeros((1,1280))

            for f in feature_dataset:
                sum_feature_dataset += f

            mean_feature = sum_feature_dataset/len(feature_dataset)

            mean_fetaure_list.append(mean_feature)
            image_list.append(images[i])
        tf.keras.backend.clear_session()


Num GPUs Available:  1
Logical devices: [LogicalDevice(name='/device:CPU:0', device_type='CPU'), LogicalDevice(name='/device:GPU:0', device_type='GPU')]


2024-11-25 21:14:17.897835: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9497 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5


Model loaded successfully.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of devices: 1
r01c01ch3.tiff


/home/gauss/miniconda3/envs/tf24/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['input_layer']. Received: the structure of inputs=*
  warnings.warn(
2024-11-25 21:14:55.176685: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


r01c02ch3.tiff


...

r03c01ch3.tiff


2024-11-25 21:37:02.716113: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


r03c02ch3.tiff


2024-11-25 21:37:58.776716: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


r03c03ch3.tiff


In [4]:
#Convert mean feature tensor array to numpy array
feature_list = [tensor.numpy() for tensor in mean_fetaure_list]
#image_num_list = [tensor.numpy() for tensor in image_list]

In [5]:
print(len(feature_list))
print(len(image_list))

382
382


In [6]:
# Create a 2D numpy array from the feature list
feature_array = np.vstack(feature_list)

# Create a DataFrame with column names
df = pd.DataFrame(feature_array, columns=['feature{}'.format(i+1) for i in range(1280)])

# Add the image numbers as a column at the beginning of the DataFrame
df.insert(0, 'Image', image_list)

# Display the DataFrame
df


,Image,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature1271,feature1272,feature1273,feature1274,feature1275,feature1276,feature1277,feature1278,feature1279,feature1280
0,r01c01ch1.tiff,-0.156932,0.406315,0.479556,-0.127781,-0.238550,-0.029096,-1.042283,0.503369,0.131065,...,-0.118518,0.141696,-0.192464,-0.123244,-0.263600,0.263082,-0.026829,-0.039631,-0.208149,-0.531510
1,r01c02ch1.tiff,-384.319855,-167.104294,-162.832565,-230.087662,-116.051941,-183.863556,-68.902000,-63.831264,-182.192947,...,-49.533752,-246.366348,-242.720932,13.577159,-255.391174,-30.350407,-243.375519,-255.174088,-298.824554,-1.556125
2,r01c03ch1.tiff,-2.326598,-1.713534,-0.480358,-0.973820,-1.659741,-0.527622,0.275194,-0.901111,-1.605661,...,-0.011723,-1.361084,-1.212016,-2.048548,-1.486497,-0.820374,-1.568422,-1.699228,-0.880083,1.357205
3,r01c04ch1.tiff,-1390.276611,-813.809509,-715.723633,-655.076416,-370.633240,-631.082275,239.501282,166.580414,-987.246216,...,334.923035,-831.624634,-851.755127,-84.618210,-996.564453,23.368824,-747.145935,-711.301453,-946.472107,689.211609
4,r01c05ch1.tiff,-9.103563,-4.001575,-3.444406,-1.096466,-3.268322,-0.743657,-1.376633,-1.651832,-4.822877,...,-2.766373,-2.866422,-6.959863,-3.470457,-7.867973,-1.809522,-12.412378,-4.251513,-4.847603,4.519289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,r16c19ch1.tiff,-2.291248,-1.191285,-0.024314,-2.583364,-3.373387,1.242514,-2.081488,-2.550482,-0.979944,...,-2.406532,-0.839835,-3.000430,-2.488299,-1.837697,-0.514891,-1.209329,-2.406834,-2.094499,-0.864419
378,r16c20ch1.tiff,-2.727712,-1.504931,-0.752008,-1.955680,-3.429655,1.026136,-1.410741,-1.267528,-1.444027,...,-2.592435,0.950363,-4.311350,-2.992813,-1.955892,-1.021201,-3.674469,-2.730391,-1.664218,1.391630
379,r16c21ch1.tiff,-1.549038,-0.508664,0.113321,-0.097479,-0.268248,-0.915819,-0.319843,0.604744,-0.527016,...,0.212185,-0.301198,-0.664778,-0.712825,-0.951846,-0.138611,-1.554838,-0.596454,-0.507400,0.383457
380,r16c22ch1.tiff,-4.028339,-0.395548,-1.167377,-1.407940,-1.568033,-0.934745,-1.292695,0.108526,-1.378940,...,-0.264225,-1.280345,-2.234407,-0.178675,-3.080866,0.340836,-2.359699,-2.490536,-1.906216,1.176826


In [7]:
#save dataframe as excel 
df.to_excel("/home/gauss/Desktop/Revised_Data_Features/EfficientNetB0 trained binary MHB Bbox Size 90/Plate_7.xlsx")


In [ ]:
len(image_list)

In [ ]:
image_list[0:150]